In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import ptitprince as pt
import seaborn as sns
import time

from itertools import combinations
from math import log
from scipy.stats import linregress, pearsonr, skew, spearmanr, zscore
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict, cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In /Users/amy/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/amy/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/amy/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/amy/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/amy/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-d

### Load data

In [2]:
#data files:
data_path = "../data/"
anat_vol_file = "PIP_n324_Freesurfer_aseg.csv"
anat_SA_file = "PIP_n324_Freesurfer_aparc2009_SA.csv"
anat_thickness_file = "PIP_n324_Freesurfer_aparc2009_thickness.csv"
rs_FC_file = "PIP_n324_rs-FC_FD_residuals.csv"
covariates_file = "PIP_n324_Framingham.csv"
data_file_names = [rs_FC_file, anat_SA_file, anat_thickness_file, anat_vol_file, covariates_file]
demographics_file = "PIP_n324_demographics.csv"
subj_list = "subj_list_n324.csv"

In [3]:
subject_list = pd.read_csv(data_path + subj_list, header=None)
str_subject_list = subject_list.astype(str)
n_subj = subject_list[0].count()

In [4]:
def extract_outcome_measure(n_subj, subject_list, outcome_file, ID_column_name, outcome_column_name):

    subjects = subject_list.values.reshape(n_subj,)
    tmp_df = pd.read_csv(outcome_file, usecols=[ID_column_name, outcome_column_name])
    y_tmp = tmp_df.loc[tmp_df[ID_column_name].isin(subjects)]
    y = y_tmp[outcome_column_name].values
    print(outcome_column_name + " loaded")

    return y, y_tmp #df coming back with labels

In [5]:
#extract response variable - IMT
outcome_column_name = "mavgimt"
ID_column_name = "id"
y, y_df = extract_outcome_measure(n_subj, str_subject_list, data_path + demographics_file, 
                                  ID_column_name, outcome_column_name)

mavgimt loaded


In [6]:
def load_combined_data(data_path, filename):

    if 'rs-FC' in filename:
        X_df = pd.read_csv(data_path + filename, header=None)
    else:
        X_df_temp = pd.read_csv(data_path + filename) #header=None
        X_df = X_df_temp.drop(X_df_temp.columns[0], axis=1)
    
    X = X_df.values
    print(filename + " loaded")

    return X

In [7]:
n_sources = 5
n_folds = 5
n_test_set = int(n_subj/n_folds)
n_train_set = n_subj - n_test_set
RANDOM_STATE = 2

In [8]:
rs_FC_data = load_combined_data(data_path, rs_FC_file)
anat_SA_data = load_combined_data(data_path, anat_SA_file)
anat_thickness_data = load_combined_data(data_path, anat_thickness_file)
anat_vol_data = load_combined_data(data_path, anat_vol_file)
data_files = [rs_FC_data, anat_SA_data, anat_thickness_data, anat_vol_data]

PIP_n324_rs-FC_FD_residuals.csv loaded
PIP_n324_Freesurfer_aparc2009_SA.csv loaded
PIP_n324_Freesurfer_aparc2009_thickness.csv loaded
PIP_n324_Freesurfer_aseg.csv loaded


In [9]:
#extract framingham indices - TotalPoints
ID_column_name = "id"

TotalPoints_column_name = "TotalPoints"
TotalPoints_tempX, TotalPoints_df_temp = extract_outcome_measure(n_subj, str_subject_list, data_path+covariates_file, 
                                                      ID_column_name, TotalPoints_column_name)


TotalPoints loaded


In [10]:
#sort df framingham data by ID
TotalPoints_df = TotalPoints_df_temp.sort_values(by=[ID_column_name])

#get sorted numpy data from df
TotalPoints_temp = TotalPoints_df[TotalPoints_column_name].values


In [11]:
#format missing data
i_nan = np.where(TotalPoints_temp == ' ')
np.put(TotalPoints_temp, i_nan, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
TotalPoints = TotalPoints_temp.astype(np.float)

In [12]:
#create covariate channel
cov_data = TotalPoints.reshape(-1, 1)
cov_data.shape

(324, 1)

In [13]:
data_files = [rs_FC_data, anat_SA_data, anat_thickness_data, anat_vol_data, cov_data]

### Functions

In [14]:
def calculate_bic(y, y_hat, n, k):
    
    #calculates BIC (Bayesian Information Criterion)
    
    residuals = y - y_hat
    rss = np.sum(residuals**2)
    #print("rss:", rss, ", n:", n, ", k:", k)
    bic = (n * log(rss/n)) + (k * log(n)) #gaussian special case formula
    
    return bic

In [15]:
def evaluate_model(y_train, y_test, y_predicted_test, num_params, intercept):
    
    #checks for skew and then uses the appropriate evaluation metric depending on presence of skew
    skewness = skew(y_train) #perform skew test on same dataset as cv, so that same overall correlation test run
    skewed = False
    
    if (skewness < 2) and (skewness > -2):
        r,p = pearsonr(y_test, y_predicted_test)
    else: #skewed
        skewed = True
        r,p = spearmanr(y_test, y_predicted_test)
    r2 = r2_score(y_test, y_predicted_test)
    
    #calculate BIC for full model
    bic_full = calculate_bic(y_test, y_predicted_test, n_test_set, num_params)
    #print("Full model bic:", bic_full )
    #calculate BIC for intercept model
    bic_intercept = calculate_bic(y_test, intercept, n_test_set, 1) 
    #print("Intercept model bic:", bic_intercept)
    #calculate Bayes factor:
    bf = bic_full/bic_intercept
    #print("Bayes Factor:", bf)

    return r, r2, bf, bic_full

In [16]:
def cross_validation(model, n_folds, X_train, y_train):

    #print("cross_validation")
    kfold = KFold(n_splits=n_folds, random_state=RANDOM_STATE, shuffle=True)
    y_predicted_cv = cross_val_predict(model, X_train, y_train, cv=kfold, verbose=0) 

    cv_scores = cross_validate(model, X_train, y_train, cv=kfold, 
                               scoring=('r2', 'neg_mean_absolute_error'), 
                               return_train_score=True)
    #print("cv_scores:", cv_scores)
    train_r2_mean = cv_scores['train_r2'].mean()
    test_r2_mean = cv_scores['test_r2'].mean()
    train_mae_mean = cv_scores['train_neg_mean_absolute_error'].mean()
    test_mae_mean = cv_scores['test_neg_mean_absolute_error'].mean()
    cv_means = {'train_r2_mean': train_r2_mean,
                'test_r2_mean': test_r2_mean,
                'train_mae_mean': train_mae_mean,
                'test_mae_mean': test_mae_mean}
    #print("cv_means:", cv_means)
    
    r2_train_set = r2_score(y_train, y_predicted_cv) 
    #print("r2_train_set:", r2_train_set)
    
    return cv_means, y_predicted_cv, kfold, r2_train_set

In [17]:
def run_split_prediction(X_train, X_test, y_train, y_test, n_folds, source_C):

    # set up pipeline
    var_thr = VarianceThreshold() 
    normalize = StandardScaler()
    svr = SVR(kernel='linear', C=source_C)
    pipeline_list = [('var_thr', var_thr), ('normalize', normalize), ('svr', svr)]
    pipe = Pipeline(pipeline_list)
    pipe.fit(X_train, y_train)
    
    y_predicted_test = pipe.predict(X_test) 
    
    svr_coef = pipe['svr'].coef_
    num_params = (pipe['svr'].coef_).size + 1
    svr_intercept = pipe['svr'].intercept_
    
    # run cross-validation
    step1_cv_means, y_predicted_cv, k_fold, r2_train_set = cross_validation(pipe, n_folds, X_train, y_train)

    return step1_cv_means, y_predicted_cv, y_predicted_test, k_fold, r2_train_set, svr_coef, num_params, svr_intercept

In [18]:
def tune_and_train_rf(X_train, y_train, n_folds):

    #Uses oob estimates to find optimal max_depth between None + 0...20
    #Refits with best max_depth
    oob_score = []
    cv_list = [None] + list(range(1, 20))
    for md in cv_list:
        rf = RandomForestRegressor(n_estimators=100, max_depth=md, oob_score=True, 
                                   random_state=RANDOM_STATE, n_jobs=-1)
        rf.fit(X_train, y_train)
        oob_score.append(rf.oob_score_) #r2
    best_max_depth = cv_list[np.argmax(oob_score)]
    rf_opt = RandomForestRegressor(n_estimators=100, max_depth=best_max_depth, 
                                   oob_score=True, random_state=RANDOM_STATE, n_jobs=-1)

    # CV
    step2_cv_means, y_predicted_cv2, k_fold, r2_train = cross_validation(rf_opt, n_folds, X_train, y_train)
    
    # refit
    rf_opt.fit(X_train, y_train)
    return rf_opt, step2_cv_means, y_predicted_cv2

In [19]:
def second_level_RF(stacked_predictions_train, stacked_predictions_test, y_train, y_test, n_folds):
    
    rf, step2_cv_means, y_predicted_cv2 = tune_and_train_rf(stacked_predictions_train, y_train, n_folds)
    y_predicted_test2 = rf.predict(stacked_predictions_test)
    
    regression = LinearRegression()
    regression.fit(y_predicted_cv2.reshape(-1,1), y_train.reshape(-1,1)) 
    regression.predict(y_predicted_test2.reshape(-1,1)) 
    reg_num_params = (regression.coef_).size
    reg_intercept = regression.intercept_
    #print("num params:", reg_num_params, "intercept:", reg_intercept)
    
    r_test2, r2_test2, bf_test2, bic_test2 = evaluate_model(y_train, y_test, y_predicted_test2, 
                                                            reg_num_params, reg_intercept)
    
    return r_test2, r2_test2, bf_test2, bic_test2, y_predicted_cv2, y_predicted_test2 

In [20]:
#with cv
def run_cov_regression(X_train, X_test, y_train, y_test):

    # set up pipeline
    var_thr = VarianceThreshold() 
    normalize = StandardScaler()
    lr = LinearRegression()
    pipeline_list = [('var_thr', var_thr), ('normalize', normalize), ('lr', lr)]
    pipe = Pipeline(pipeline_list)
    pipe.fit(X_train, y_train)
    y_predicted_test = pipe.predict(X_test)
    
    lr_coef = pipe['lr'].coef_
    lr_intercept = pipe['lr'].intercept_
    
    # run cross-validation
    step1_cv_means, y_predicted_cv, k_fold, r2_train_set = cross_validation(pipe, n_folds, X_train, y_train)

    return step1_cv_means, y_predicted_cv, y_predicted_test, k_fold, lr_coef, lr_intercept

### monte carlo



#### 5 channel cv model

In [21]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
cov_data_imputed = imputer.fit_transform(cov_data)

#### channel combinations

In [22]:
data_files = [rs_FC_data, anat_SA_data, anat_thickness_data, anat_vol_data, cov_data_imputed]
data_file_names = [rs_FC_file, anat_SA_file, anat_thickness_file, anat_vol_file, covariates_file]

n_channels = len(data_files)
channel_subsets = []
    
for k in np.arange(2, n_channels +1):    
    channel_subsets += list(combinations(np.arange(n_channels), k))

num_comb = len(channel_subsets)

In [23]:
channel_subsets_v2 = []
    
for k in np.arange(2, n_channels +1):    
    channel_subsets_v2 += list(combinations(np.arange(1,n_channels+1), k))
    
channel_subsets_v2

[(1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5),
 (1, 2, 3, 4),
 (1, 2, 3, 5),
 (1, 2, 4, 5),
 (1, 3, 4, 5),
 (2, 3, 4, 5),
 (1, 2, 3, 4, 5)]

In [24]:
str_subset_v2 = []

for i, subset in enumerate(channel_subsets_v2):
    str_subset_v2.append(str(subset))
    
str_subset_v2

['(1, 2)',
 '(1, 3)',
 '(1, 4)',
 '(1, 5)',
 '(2, 3)',
 '(2, 4)',
 '(2, 5)',
 '(3, 4)',
 '(3, 5)',
 '(4, 5)',
 '(1, 2, 3)',
 '(1, 2, 4)',
 '(1, 2, 5)',
 '(1, 3, 4)',
 '(1, 3, 5)',
 '(1, 4, 5)',
 '(2, 3, 4)',
 '(2, 3, 5)',
 '(2, 4, 5)',
 '(3, 4, 5)',
 '(1, 2, 3, 4)',
 '(1, 2, 3, 5)',
 '(1, 2, 4, 5)',
 '(1, 3, 4, 5)',
 '(2, 3, 4, 5)',
 '(1, 2, 3, 4, 5)']

In [27]:
#100 runs - on channel combinations

n_sources = 5
data_files = [rs_FC_data, anat_SA_data, anat_thickness_data, anat_vol_data, cov_data_imputed]
data_file_names = [rs_FC_file, anat_SA_file, anat_thickness_file, anat_vol_file, covariates_file]

n_splits = 100
rs = ShuffleSplit(n_splits=n_splits, test_size=n_test_set, train_size=n_train_set, random_state=RANDOM_STATE)

mc_y_train = np.zeros(shape=(n_splits,n_train_set))
mc_y_test = np.zeros(shape=(n_splits,n_test_set))

i_split = 0
step1_corrs = np.zeros(shape=(n_splits, n_sources))
step1_r2s = np.zeros(shape=(n_splits, n_sources))
step1_bfs = np.zeros(shape=(n_splits, n_sources))
step1_bics = np.zeros(shape=(n_splits, n_sources))

mc_stacked_predictions_train = np.zeros(shape=(n_splits, n_sources, n_train_set)) 
mc_stacked_predictions_test = np.zeros(shape=(n_splits, n_sources, n_test_set)) 

comb_r = np.zeros(shape=(n_splits,num_comb))
comb_r2 = np.zeros(shape=(n_splits,num_comb))
comb_bf = np.zeros(shape=(n_splits,num_comb))
comb_bic = np.zeros(shape=(n_splits,num_comb))

comb_y_pred_cv = np.zeros(shape=(n_splits,num_comb,n_train_set))
comb_y_pred_test = np.zeros(shape=(n_splits,num_comb,n_test_set))


for train_index, test_index in rs.split(np.zeros(n_subj)):
    print("Split:", i_split)
    
    #first level
    stacked_predictions_train = np.zeros(shape=(n_train_set, n_sources)) 
    stacked_predictions_test = np.zeros(shape=(n_test_set, n_sources))
    source_index = 0
    source_C = 10 ** -3
    
    y_train, y_test = y[train_index], y[test_index]
    mc_y_train[i_split,:] = y_train
    mc_y_test[i_split,:] = y_test

    for source_name, source in zip(data_file_names, data_files):
        if 'aseg' in source_name:
            source_C = 1
        X_train, X_test = source[train_index], source[test_index]
        
        if 'Framingham' in source_name:
            #linear regression
            step1_cv_means, y_predicted_cv, y_predicted_test, k_fold, \
                lr_coef, lr_intercept = run_cov_regression(X_train, 
                                                           X_test, 
                                                           y_train, 
                                                           y_test)
            #dummy values to exclude lr
            svr_num_params = -1
            svr_intercept = -99
        else:
            step1_cv_means, y_predicted_cv, y_predicted_test, k_fold, \
                step1_r2_train_set, svr_coef, svr_num_params, \
                svr_intercept = run_split_prediction(X_train, X_test, 
                                                     y_train, y_test, 
                                                     n_folds, source_C)
            #svr_coefs.append(svr_coef)
        
        #compute correlation coefficient, coefficient of determination and bayes factor on test set:
        step1_r_test_set, step1_r2_test_set, step1_bf, \
            step1_bic = evaluate_model(y_train, y_test, y_predicted_test, 
                                       svr_num_params, svr_intercept)
        step1_corrs[i_split,source_index] = step1_r_test_set 
        step1_r2s[i_split,source_index] = step1_r2_test_set
        step1_bfs[i_split,source_index] = step1_bf
        step1_bics[i_split,source_index] = step1_bic
        #prediction stacking
        stacked_predictions_train[:,source_index] = y_predicted_cv
        stacked_predictions_test[:,source_index] = y_predicted_test
        
        source_index += 1
        
    mc_stacked_predictions_train[i_split,:,:] = stacked_predictions_train.T
    mc_stacked_predictions_test[i_split,:,:] = stacked_predictions_test.T
    
    #second level
    for comb_i, subset in enumerate(channel_subsets):
        #str_subset.append(str(subset))
        print("combination:", comb_i)
        stacked_train_combination = stacked_predictions_train[:,list(subset)]
        stacked_test_combination = stacked_predictions_test[:,list(subset)]
        #stacked_r2s = stacked_step1_r2_test[:,list(subset)]
        stacked_r2s = step1_r2s[i_split,list(subset)]
        #print(subset, stacked_r2s)
        comb_r[i_split,comb_i], comb_r2[i_split,comb_i], comb_bf[i_split,comb_i], \
            comb_bic[i_split,comb_i], y_predicted_cv2, \
            y_predicted_test2 = second_level_RF(stacked_train_combination, 
                                                stacked_test_combination, 
                                                y_train, y_test, n_folds)
        comb_y_pred_cv[i_split,comb_i,:] = y_predicted_cv2
        comb_y_pred_test[i_split,comb_i,:] = y_predicted_test2

    
    i_split += 1


print(" ")
print(" ")
print("MONTE CARLO:")
print(" ")
print("Step 1: SVR (brain) / LR (FI)")
print("correlations, r:", step1_corrs)
print("average test set r:", np.mean(step1_corrs,axis=0))
print("coefficient of determination (r2):", step1_r2s)
print("average test set coefficient of determination (r2):", np.mean(step1_r2s,axis=0))
print("bayes factor:", step1_bfs)
print("average test set bayes factor:", np.mean(step1_bfs,axis=0))
print("bic:", step1_bics)
print("average test set bic:", np.mean(step1_bics,axis=0))
print(" ")
print("Step 2: RF")
print("correlations, r:", comb_r)
print("average test set r:", np.mean(comb_r))
print("coefficient of determination (r2):", comb_r2)
print("average test set coefficient of determination (r2):", np.mean(comb_r2))
print("bayes factor:", comb_bf)
print("average test set bayes factor:", np.mean(comb_bf))
print("bic:", comb_bic)
print("average test set bic:", np.mean(comb_bic))



Split: 0
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 1
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 2
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination

combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 20
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 21
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 22
combination: 0
combination: 1
combination: 2
c

combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 40
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 41
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combinat

combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 60
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 61
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 1

combination: 23
combination: 24
combination: 25
Split: 79
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 80
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 81
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combi

combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
Split: 99
combination: 0
combination: 1
combination: 2
combination: 3
combination: 4
combination: 5
combination: 6
combination: 7
combination: 8
combination: 9
combination: 10
combination: 11
combination: 12
combination: 13
combination: 14
combination: 15
combination: 16
combination: 17
combination: 18
combination: 19
combination: 20
combination: 21
combination: 22
combination: 23
combination: 24
combination: 25
 
 
MONTE CARLO:
 
Step 1: SVR (brain) / LR (FI)
correlations, r: [[ 0.08475129  0.23294175  0.13057923  0.02507871  0.40867415]
 [ 0.01070141  0.26001133  0.20105747  0.25520934  0.45158251]
 [ 0.10164878  0.00190889  0.07315493  0.11766366  0.56234964]
 [-0.1173296  -0.09608485  0.0273245   0.15132741  0.28694679]
 [ 0.13173411  0.2263729   0.26106778  0.02844989  0.40678426]
 [ 0.25466845  0.06085926  0.04043952  0.12

In [28]:
#calculate RMSE from predictions

def calculate_rmse(y,y_pred,n):
    
    mse = (1/n)*np.sum(np.square(y-y_pred))
    rmse = np.sqrt(mse)
    
    return rmse


### save single channel results

In [29]:
mc_stacked_predictions = np.concatenate((mc_stacked_predictions_train,mc_stacked_predictions_test),axis=2)
mc_stacked_predictions.shape

(100, 5, 324)

In [89]:
# single channel stacked predictions 
f = open('../results/checked/mc_single_channel_predictions.pckl', 'wb')
pickle.dump([mc_stacked_predictions],f) 
f.close()

In [90]:
# single channel r, r2 (test set)
f = open('../results/mc_single_channel_r_r2_test.pckl', 'wb')
pickle.dump([step1_corrs, step1_r2s],f) 
f.close()

In [91]:
# single channel bfs
f = open('../results/mc_single_channel_bfs.pckl', 'wb')
pickle.dump([step1_bfs],f)
f.close()

In [92]:
# step1_bic 
f = open('../results/mc_single_channel_bic.pckl', 'wb')
pickle.dump([step1_bics],f)
f.close()

In [47]:
single_channel_rmse = np.zeros(shape=(n_splits,n_sources))
split = 0
for x in mc_stacked_predictions: #method of iterating through numpy arrays
    channel = 0
    for y_pred1 in x:
        single_channel_rmse[split,channel] = calculate_rmse(mc_y[split,:], y_pred1, n_subj)
        channel += 1
    split += 1

In [93]:
# single channel stacked predictions rmse
f = open('../results/mc_single_channel_predictions_rmse.pckl', 'wb')
pickle.dump([single_channel_rmse],f)
f.close()

### save combination channel results

In [45]:
comb_y_pred = np.concatenate((comb_y_pred_cv,comb_y_pred_test),axis=2)
comb_y_pred.shape

(100, 26, 324)

In [46]:
mc_y = np.concatenate((mc_y_train, mc_y_test),axis=1)
mc_y.shape

(100, 324)

In [95]:
# y_pred(100*26*324), y(100*324) (+ str_subset for channel combination order)
f = open('../results/mc_comb_y_predicted.pckl', 'wb')
pickle.dump([comb_y_pred, mc_y, str_subset_v2],f)
f.close()

In [96]:
#use pickle to save mc results for channel combinations

# r, r2, bf (+ str_subset for channel combination order)
f = open('../results/mc_comb_metrics.pckl', 'wb')
pickle.dump([comb_r, comb_r2, comb_bf, str_subset_v2],f)
f.close()


In [97]:
# save mc bic's using pickle

# comb_bic + str_subset
f = open('../results/mc_comb_bic.pckl', 'wb')
pickle.dump([comb_bic, str_subset_v2],f)
f.close()


In [64]:
rmse = np.zeros(shape=(n_splits,num_comb))
split = 0
for x in comb_y_pred: #method of iterating through numpy arrays
    comb = 0
    for y_pred in x: 
        rmse[split,comb] = calculate_rmse(mc_y[split,:], y_pred, n_subj)
        comb += 1
    split += 1
        

In [98]:
# rmse (+ str_subset for channel combination order)
f = open('../results/mc_comb_rmse.pckl', 'wb')
pickle.dump([rmse, str_subset_v2],f)
f.close()

### Brain age

In [5]:
outcome_column_name = "age"
ID_column_name = "id"
age, age_df = extract_outcome_measure(n_subj, str_subject_list, data_path + demographics_file, 
                                      ID_column_name, outcome_column_name)

age loaded


In [16]:
#FIRST LEVEL
n_sources = 4
data_files = [rs_FC_data, anat_SA_data, anat_thickness_data, anat_vol_data]
data_file_names = [rs_FC_file, anat_SA_file, anat_thickness_file, anat_vol_file]

start_time = time.time()
print("Starting first level...")
stacked_predictions_train = np.zeros(shape=(n_train_set, n_sources)) 
stacked_predictions_test = np.zeros(shape=(n_test_set, n_sources))
source_index = 0
source_C = 10 ** -3

stacked_step1_r2_test = np.zeros(shape=(1,n_sources))
svr_coefs = []

for source_name, source in zip(data_file_names, data_files):
    print("source", source_index + 1, ": ", source_name)
    if 'aseg' in source_name:
        source_C = 1
    X = source
    X_train, X_test, age_train, age_test = train_test_split(X, age, test_size=n_test_set, random_state=RANDOM_STATE)
    step1_cv_means, age_predicted_cv, age_predicted_test, k_fold, step1_r2_train_set, svr_coef, svr_num_params, svr_intercept = run_split_prediction(X_train, X_test, age_train, age_test, n_folds, source_C)

    svr_coefs.append(svr_coef)
    #compute correlation coefficient and coefficient of determination on test set:
    step1_r_test_set, step1_r2_test_set, step1_bf, step1_bic = evaluate_model(age_train, age_test, age_predicted_test, svr_num_params, svr_intercept)
    print("test set r:", step1_r_test_set)
    print("test set coefficient of determination (r2):", step1_r2_test_set)
    #prediction stacking
    stacked_predictions_train[:,source_index] = age_predicted_cv 
    stacked_predictions_test[:,source_index] = age_predicted_test
    stacked_step1_r2_test[0,source_index] = step1_r2_test_set
    source_index += 1
    
print("First level complete")
end_time = time.time()
elapsed_time = end_time - start_time
print("time elapsed:", str(datetime.timedelta(seconds=elapsed_time)))

Starting first level...
source 1 :  PIP_n324_rs-FC_FD_residuals.csv
test set r: 0.08816093762722894
test set coefficient of determination (r2): -0.1673306219092381
source 2 :  PIP_n324_Freesurfer_aparc2009_SA.csv
test set r: 0.04827340013406542
test set coefficient of determination (r2): -0.00813284407303394
source 3 :  PIP_n324_Freesurfer_aparc2009_thickness.csv
test set r: 0.3334667143044422
test set coefficient of determination (r2): 0.0955249401247068
source 4 :  PIP_n324_Freesurfer_aseg.csv
test set r: 0.522852274356508
test set coefficient of determination (r2): 0.17015368210437354
First level complete
time elapsed: 0:00:37.116545


In [17]:
#SECOND LEVEL
start_time = time.time()
print("Starting second level...")
r_test2, r2_test2, bf_test2, bic_test2, age_predicted_cv2, age_predicted_test2 = second_level_RF(stacked_predictions_train, 
                                                                                                 stacked_predictions_test, 
                                                                                                 age_train, age_test, n_folds)
print("test set r:", r_test2)
print("test set coefficient of determination (r2):", r2_test2)
print("Second level complete")
end_time = time.time()
elapsed_time = end_time - start_time
print("time elapsed:", str(datetime.timedelta(seconds=elapsed_time)))

Starting second level...
test set r: 0.524635172299574
test set coefficient of determination (r2): 0.27320428586799506
Second level complete
time elapsed: 0:00:12.666956


In [18]:
age_observed = np.concatenate((age_train, age_test))
age_predicted_overall = np.concatenate((age_predicted_cv2, age_predicted_test2))

# save results with pickle to be able to load into paper figures
f = open('../results/brain_age.pckl', 'wb')
pickle.dump([age_observed, age_predicted_overall],f)
f.close()